# UNN NLP 2021 2nd Lab

In [ ]:
!pip install transformers

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    TrainingArguments,
    Trainer,
    pipeline)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
%cd '/content/gdrive/My Drive/UNN-2021-2-nd-laboratory-work/'

Mounted at /content/gdrive/
/content/gdrive/My Drive/UNN-2021-2-nd-laboratory-work


### Описание датасета

Датасет представляет собой набор из 124167 анекдотов.

In [ ]:
data_raw = open("anek.txt", "r", encoding='utf-8')
data_raw = data_raw.read()
data_raw = data_raw.replace('\n\n','\n') 
data_raw = data_raw.split(r"<|startoftext|>")

75% анекдотов используем для тренировочной выборки и 25% для тестовой.

In [ ]:
#data = open("train.txt", "w+")
#for i in range(93125):
#  data.write(data_raw[i])
#data.close()

In [ ]:
#data = open("test.txt", "w+")
#for i in range(93125, len(data_raw)):
#  data.write(data_raw[i])
#data.close()

Пример используемых анекдотов:

In [ ]:
for i in range(5,11):
    print(data_raw[i])

Поссорилась с мужем. Пока он спал, я мысленно развелась с ним, поделила имущество, переехала, поняла, что жить без него не могу, дала последний шанс, вернулась. В итоге, ложусь спать уже счастливой женщиной.

Если тебя посещают мысли о смерти - это еще полбеды. Беда - это когда смерть посещают мысли о тебе...

Все мне говорили, что у меня ничего не получится, но я не сдался и доказал, что они правы.

Распорядок работы душевой: в понедельник, среду и пятницу запускаются мужчины. Во вторник, четверг и субботу - женщины. В воскресенье все выпускаются.

Если в вашем холодильнике нет ни одного подгнивающего овоща, купленного в надежде на правильное питание, проверьте, человек ли вы.

Переписываются молодой парень с блондинкой:- Вы знаете, я очень люблю Кафку, а вы?- Я тоже люблю кафку. Офобенно пфонную.



In [ ]:
train_path = 'train.txt'
test_path = 'test.txt'

### Модель


В качестве модели используестся `rugpt3small_based_on_gpt` сбербанка. Модели боольшего размера не удалось иcпользовать на google colab-e. `rugpt3small_based_on_gpt` - модель, предназначенная для задач языкового моделирования.

Выполним загрузку модели:

In [ ]:
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

### Токенизатор


Для токенизации будем использовать то же токенизатор, который использовался для модели `rugpt3small_based_on_gpt` cбербанка.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Downloading:   0%|          | 0.00/608 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


DataCollator используется для формирования пакета из набора обучающих и тестовых данных. DataCollator может применять некоторую обработку данных(например, заполнение), если данные не имеют одинаковой длины.

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=64)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_path,
    block_size=64)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


### Точная настройка предварительно обученной модели.

In [ ]:
training_args = TrainingArguments(
    output_dir = 'data/out',
    overwrite_output_dir = True,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    learning_rate = 5e-5,
    num_train_epochs = 3,
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 42214
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3960


Step,Training Loss
500,3.704900
1000,3.635400
1500,3.532500
2000,3.407200
2500,3.401300
3000,3.304000
3500,3.265500


Saving model checkpoint to data/out/checkpoint-500
Configuration saved in data/out/checkpoint-500/config.json
Model weights saved in data/out/checkpoint-500/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-1000
Configuration saved in data/out/checkpoint-1000/config.json
Model weights saved in data/out/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-1500
Configuration saved in data/out/checkpoint-1500/config.json
Model weights saved in data/out/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-2000
Configuration saved in data/out/checkpoint-2000/config.json
Model weights saved in data/out/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-2500
Configuration saved in data/out/checkpoint-2500/config.json
Model weights saved in data/out/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-3000
Configuration saved in data/out/checkpoint-3000/config.json
M

TrainOutput(global_step=3960, training_loss=3.441420491536458, metrics={'train_runtime': 4159.5695, 'train_samples_per_second': 30.446, 'train_steps_per_second': 0.952, 'total_flos': 4136318189568000.0, 'train_loss': 3.441420491536458, 'epoch': 3.0})

In [ ]:
trainer.save_model()

Saving model checkpoint to data/out
Configuration saved in data/out/config.json
Model weights saved in data/out/pytorch_model.bin


In [ ]:
ppln = pipeline('text-generation', tokenizer='sberbank-ai/rugpt3small_based_on_gpt2', model='data/out')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Генерация текста

`max_length(int)` длина создаваемой последовательности.

`repetition_penalty(float)` параметр штрафа за повторение. 1.0 означает отсутствие штрафа. Он может быть довольно эффективным для предотвращения повторений, но, кажется, очень чувствителен к различным моделям и вариантам использования.

`do_sample` если установлено значение True, осуществляется случайный выбор следующего слова в соответствии с его условным распределением вероятностей.

`temperature = n`  этот параметр контролирует случайность сгенерированного текста. Обычно temperature 0.70–0.90 является наиболее распространенной для задач генерации текста. Также, чем ближе к нулю значение, тем больше генерация будет похожа на жадный подбор слов.

`top_k = n` определяет n кол-во слов, которые обладают наибольшей вероятностью из условного распределения вероятностей всех слов, что сужает выбор для модели и отбрасывает максимально неподходящие слова сразу.

`top_p = n` определяет n слов, чья вероятностная масса вместе равна n%. Например, значение top_p, равное 0.9, будет означать, что из выборки будут исключены самые маловероятные 10% всех слов.

Хотя в теории top_p кажется более элегантным, чем top_k , на практике оба метода работают хорошо. top_p также можно использовать в сочетании с top_k , что позволяет избежать слов с очень низким рейтингом, но при этом допускает некоторый динамический выбор.


In [ ]:
print(ppln('Купил мужик шляпу')[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Купил мужик шляпу, а она оказалась не той.
- А что такое "плох


Так как используемый датасет представляет собой набор анекдотов, никак не связанных друг с другом, будем генерировать короткие тексты, примерно равные длине одного анекдота. 

In [ ]:
print(ppln('Британские ученые', max_length=32)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Британские ученые установили, что если в день рождения подарить жене бриллиантовое колье, то она будет счастлива.
- А что такое "плох


In [ ]:
print(ppln('Британские ученые', max_length=32, repetition_penalty=2.5)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Британские ученые установили, что если в день рождения подарить жене бриллиантовое колье и кольцо с брильянтами - она будет счастлива.
-


In [ ]:
print(ppln('После долгих экспериментов свинья-алхимик', max_length=32, do_sample=True, top_k=40)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


После долгих экспериментов свинья-алхимик пришла к выводу, что если все правильно и будет получаться, то получится не то, что хотела.
Чем больше


In [ ]:
print(ppln('- Пап, а о чем ты мечтаешь?', max_length=32, do_sample=True, top_p=0.9)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- Пап, а о чем ты мечтаешь?Голос из зала:- О мечтах, сынок!
- Эх, когда же жизнь научится так грубо


In [ ]:
print(ppln('- Что самое главное в современной экономике?', max_length=32, do_sample=True, temperature=0.6)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- Что самое главное в современной экономике?- Не в деньгах счастье, а в их количестве.
- Вы знаете, мой муж мне изменяет.- Это как?-


In [ ]:
print(ppln('- Милый, хочешь, я подарю тебе свое сердце', max_length=32, do_sample=True, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


- Милый, хочешь, я подарю тебе свое сердце?- Конечно!- Тогда подари мне его, пожалуйста!
- Мужики, я не понимаю, как


In [ ]:
print(ppln('Повар спрашивает повара', max_length=32, do_sample=True, temperature=0.75)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Повар спрашивает повара:- В чем смысл жизни?- В еде.
- Ты что, в командировке?- Да нет, просто я так вырос, что


In [ ]:
print(ppln('После долгих экспериментов свинья-алхимик', max_length=32, do_sample=True, top_k=50, top_p=0.95, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


После долгих экспериментов свинья-алхимик наконец-то нашла способ, как заставить алкаша пить.
- Я уже неделю не могу прийти домой из-


In [ ]:
print(ppln('После долгих попыток еж-алхимик', max_length=32, do_sample=True, top_k=50, top_p=0.95, temperature=0.7)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


После долгих попыток еж-алхимик таки смог убедить жену, что она - злая колдунья.
- Девушка, а вы не знаете, что такое
